In [36]:
#Import Libriaries for EDA
import numpy as np
import pandas as pd
#Install the mysql-connector-python package

!pip install mysql-connector-python
import mysql.connector




In [46]:
df=pd.read_csv('flight_delays.csv')

Step1 :Extract Data :Basic Data Exploration

In [48]:
#Displaye the first few rows of the dataset:
df.head()

,FlightID,Airline,FlightNumber,Origin,Destination,ScheduledDeparture,ActualDeparture,ScheduledArrival,ActualArrival,DelayMinutes,DelayReason,Cancelled,Diverted,AircraftType,TailNumber,Distance
0,1,United,4558,ORD,MIA,9/1/2024 8:11,9/1/2024 8:30,9/1/2024 12:11,9/1/2024 12:19,8,Weather,True,False,Boeing 737,N71066,1031
1,2,Delta,8021,LAX,MIA,9/1/2024 10:25,9/1/2024 10:41,9/1/2024 13:25,9/1/2024 13:27,2,Air Traffic Control,True,True,Airbus A320,N22657,1006
2,3,Southwest,7520,DFW,SFO,9/1/2024 16:53,9/1/2024 17:05,9/1/2024 17:53,9/1/2024 18:07,14,Weather,True,True,Boeing 737,N95611,2980
3,4,Delta,2046,ORD,BOS,9/1/2024 14:44,9/1/2024 15:04,9/1/2024 18:44,9/1/2024 18:34,-10,NaN,False,False,Boeing 777,N90029,1408
4,5,Delta,6049,LAX,SEA,9/1/2024 1:51,9/1/2024 2:08,9/1/2024 5:51,9/1/2024 6:15,24,Air Traffic Control,False,True,Boeing 737,N27417,2298


In [50]:
#Display the Column names:
df.columns

Index(['FlightID', 'Airline', 'FlightNumber', 'Origin', 'Destination',
       'ScheduledDeparture', 'ActualDeparture', 'ScheduledArrival',
       'ActualArrival', 'DelayMinutes', 'DelayReason', 'Cancelled', 'Diverted',
       'AircraftType', 'TailNumber', 'Distance'],
      dtype='object')

In [52]:
#define shape of Data
df.shape

(1048575, 16)

In [54]:
#Get information about the Dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 16 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   FlightID            1048575 non-null  int64 
 1   Airline             1048575 non-null  object
 2   FlightNumber        1048575 non-null  int64 
 3   Origin              1048575 non-null  object
 4   Destination         1048575 non-null  object
 5   ScheduledDeparture  1048575 non-null  object
 6   ActualDeparture     1048575 non-null  object
 7   ScheduledArrival    1048575 non-null  object
 8   ActualArrival       1048575 non-null  object
 9   DelayMinutes        1048575 non-null  int64 
 10  DelayReason         767243 non-null   object
 11  Cancelled           1048575 non-null  bool  
 12  Diverted            1048575 non-null  bool  
 13  AircraftType        1048575 non-null  object
 14  TailNumber          1048575 non-null  object
 15  Distance            1048575 non-

In [56]:
#Handle missing Value Look for Null Values
df.isnull().sum()

FlightID                   0
Airline                    0
FlightNumber               0
Origin                     0
Destination                0
ScheduledDeparture         0
ActualDeparture            0
ScheduledArrival           0
ActualArrival              0
DelayMinutes               0
DelayReason           281332
Cancelled                  0
Diverted                   0
AircraftType               0
TailNumber                 0
Distance                   0
dtype: int64

In [60]:
#Remove Null Values
df.dropna(inplace=True)

In [62]:
#Re- check Null Values
df.isnull().sum()

FlightID              0
Airline               0
FlightNumber          0
Origin                0
Destination           0
ScheduledDeparture    0
ActualDeparture       0
ScheduledArrival      0
ActualArrival         0
DelayMinutes          0
DelayReason           0
Cancelled             0
Diverted              0
AircraftType          0
TailNumber            0
Distance              0
dtype: int64

In [64]:
#re-check Shape of the Data( Must reduce since we remove Null values)
df.shape

(767243, 16)

In [66]:
#Get summary statistic and analyse the Data
df.describe()

,FlightID,FlightNumber,DelayMinutes,Distance
count,7.672430e+05,767243.000000,767243.000000,767243.000000
mean,5.240737e+05,4998.390060,15.493789,1550.549281
std,3.027458e+05,2885.670294,8.651730,836.617900
min,1.000000e+00,1.000000,1.000000,100.000000
25%,2.617045e+05,2498.000000,8.000000,825.000000
50%,5.241160e+05,5000.000000,15.000000,1552.000000
75%,7.861300e+05,7495.000000,23.000000,2274.000000
max,1.048574e+06,9999.000000,30.000000,3000.000000


STEP 2: Clean and Transform Data

In [68]:
#A. Convert string dates to datetime
date_cols = ['ScheduledDeparture', 'ActualDeparture', 'ScheduledArrival', 'ActualArrival']
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')  # Handle bad data with 'coerce'


In [69]:
#B. Create a new column: TotalDelay
df['TotalDelay'] = (df['ActualArrival'] - df['ScheduledArrival']).dt.total_seconds() / 60


In [70]:
#C. Handle null values in DelayReason
df['DelayReason'] = df['DelayReason'].fillna('Unknown')


In [71]:
#D. Ensure consistent formatting ,Normalization.
df.columns = [col.strip().lower() for col in df.columns]  # Normalize column names


In [76]:
df

,flightid,airline,flightnumber,origin,destination,scheduleddeparture,actualdeparture,scheduledarrival,actualarrival,delayminutes,delayreason,cancelled,diverted,aircrafttype,tailnumber,distance,totaldelay
0,1,United,4558,ORD,MIA,2024-09-01 08:11:00,2024-09-01 08:30:00,2024-09-01 12:11:00,2024-09-01 12:19:00,8,Weather,True,False,Boeing 737,N71066,1031,8.0
1,2,Delta,8021,LAX,MIA,2024-09-01 10:25:00,2024-09-01 10:41:00,2024-09-01 13:25:00,2024-09-01 13:27:00,2,Air Traffic Control,True,True,Airbus A320,N22657,1006,2.0
2,3,Southwest,7520,DFW,SFO,2024-09-01 16:53:00,2024-09-01 17:05:00,2024-09-01 17:53:00,2024-09-01 18:07:00,14,Weather,True,True,Boeing 737,N95611,2980,14.0
4,5,Delta,6049,LAX,SEA,2024-09-01 01:51:00,2024-09-01 02:08:00,2024-09-01 05:51:00,2024-09-01 06:15:00,24,Air Traffic Control,False,True,Boeing 737,N27417,2298,24.0
6,7,Southwest,4188,ORD,JFK,2024-09-01 05:47:00,2024-09-01 06:02:00,2024-09-01 10:47:00,2024-09-01 10:54:00,7,Weather,False,True,Boeing 777,N25382,1674,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048569,1048570,United,3057,ORD,BOS,2024-09-01 19:09:00,2024-09-01 19:30:00,2024-09-01 20:09:00,2024-09-01 20:25:00,16,Maintenance,False,True,Boeing 737,N54576,787,16.0
1048570,1048571,Delta,8720,ORD,SFO,2024-09-01 07:10:00,2024-09-01 07:32:00,2024-09-01 08:10:00,2024-09-01 08:29:00,19,Air Traffic Control,False,False,Boeing 777,N59866,2265,19.0
1048571,1048572,American Airlines,2532,ORD,SEA,2024-09-01 06:41:00,2024-09-01 07:02:00,2024-09-01 09:41:00,2024-09-01 10:07:00,26,Weather,False,True,Airbus A320,N44952,525,26.0
1048572,1048573,Southwest,6007,LAX,MIA,2024-09-01 09:01:00,2024-09-01 09:17:00,2024-09-01 11:01:00,2024-09-01 11:25:00,24,Weather,False,True,Airbus A320,N58488,955,24.0


In [84]:
df.to_csv('cleaned_flight_delays_v.csv', index=False)

#STEP 3: Create MySQL Table (in MySQL Workbench)
#Open MySQL Workbench and run:CREATE DATABASE IF NOT EXISTS flight_data;
#USE flight_data;

#CREATE TABLE flights (
    flightid INT,
    airline VARCHAR(100),
    flightnumber VARCHAR(50),
    origin VARCHAR(10),
    destination VARCHAR(10),
    scheduleddeparture DATETIME,
    actualdeparture DATETIME,
    scheduledarrival DATETIME,
    actualarrival DATETIME,
    delayminutes INT,
    delayreason VARCHAR(255),
    cancelled BOOLEAN,
    diverted BOOLEAN,
    aircrafttype VARCHAR(50),
    tailnumber VARCHAR(50),
    distance INT,
    totaldelay FLOAT
);


In [86]:
# STEP 4: Load into MySQL (Load Phase)
import pandas as pd

df = pd.read_csv('cleaned_flight_delays_v.csv')


In [88]:
!pip install mysql-connector-python

import mysql.connector

# MySQL connection settings
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    port="3306",
    password="Venus",
    database="flight_data"
)

cursor = conn.cursor()

# SQL insert query (match this to your table structure)
insert_query = '''
INSERT INTO flights (
    flightid, airline, flightnumber, origin, destination,
    scheduleddeparture, actualdeparture, scheduledarrival, actualarrival,
    delayminutes, delayreason, cancelled, diverted, aircrafttype,
    tailnumber, distance, totaldelay
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''

# Insert in batches
for i in range(0, len(df), 500):
    batch = df.iloc[i:i+500]
    for _, row in batch.iterrows():
        values = tuple(row)
        try:
            cursor.execute(insert_query, values)
        except Exception as e:
            print("Error inserting row:", e)
    conn.commit()

cursor.close()
conn.close()

print("Cleaned data loaded into MySQL.")


Cleaned data loaded into MySQL.
